In [1]:
import csv
lists=[]
with open('HW2/Yahoo_Movie5.csv', newline='',encoding='utf-8') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        lists.append(row)

In [2]:
import jieba
from opencc import OpenCC

cc = OpenCC('tw2sp')
text=[]
target=[]
types={'N/A':0}
type_num=0

for line in lists:
    name=cc.convert(line[0])
    discr=cc.convert(line[2])
    jieba.add_word(name)
    temp=name+discr
    words=jieba.cut(temp)
    word=''
    for w in words:
        word=word+w+' '
    text.append(word)
    if len(line)<=3:
        temp='N/A'
    else:
        temp=line[3]
    target.append(temp)
    if temp not in types:
        type_num+=1
        types[temp]=type_num

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Cindy\AppData\Local\Temp\jieba.cache
Loading model cost 0.608 seconds.
Prefix dict has been built successfully.


In [3]:
text[0]

'噤界II 故事 承接 上 一集 ， 家里 发生意外 之后 ， 一家人 （ 艾蜜莉 布朗 、 诺亚 朱佩 、 米莉森 西蒙斯 ） 现在 必须 去 面对 外界 未知 的 恐怖 世界 ， 继续 为了 生存 而 努力 噤声 。 他们 迫使 冒险 踏入 全新 未知 的 地方 ， 但 很快 就 意识 到 ， 那些 透过 声音 来 狩猎 的 生物 并 不是 潜伏 在 沙子 路上 的 唯一 威胁 。 '

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import numpy as np

document=np.array(text)
vectorizer = CountVectorizer(stop_words=None, token_pattern="(?u)\\b\\w+\\b")
tf=vectorizer.fit_transform(document)
tf_vis=pd.DataFrame(tf.toarray(),columns=vectorizer.get_feature_names())

In [5]:
transformer = TfidfTransformer(smooth_idf=True)
tfidf = transformer.fit_transform(tf)
tfidf_vis = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names())

In [6]:
l=[]
for t in target:
    l.append(types[t])
label=np.array(l)
data=tfidf.toarray()

In [7]:
train_data=data[:5070,:]
test_data=data[5070:,:]
train_label=label[:5070]
test_label=label[5070:]

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
KnnModel=knn.fit(train_data,train_label)

In [9]:
predict=KnnModel.predict(test_data)

In [10]:
croecct=0
for i in range(predict.shape[0]):
    if predict[i]==test_label[i]:
        croecct+=1

In [11]:
acc=croecct/500*100
print("準確率: "+str(acc)+"%")

準確率: 41.6%
